<a href="https://colab.research.google.com/github/GitBar20/AI/blob/main/Exper1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Формирование модели, обучение и тестирование

## Эксперимент 1
Созданим простую модель и обучим ее

In [ ]:
import time
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self
    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        print('Время обработки: {:.2f} с'.format(time.time() - self.t))

In [ ]:
def model_check(model):
  # Входная форма
  input_shape = model.input.get_shape()[1:]

  # Выходная форма
  output_shape = model.output.get_shape()[1:]

  # Сравнение
  assert input_shape == output_shape, 'Вход и выход не совпадают по форме'

In [ ]:
# Работа с операционной системой
import os

# Работа с массивами данных
import numpy as np

# Работа с таблицами
import pandas as pd

# Основные слои
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Conv2DTranspose, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization, Concatenate

# класс для использования Functional API
from tensorflow.keras import Model

# Оптимизаторы для обучения модели
from tensorflow.keras.optimizers import Adam

# Функции-утилиты
from tensorflow.keras import utils

# Готовые датасеты
from tensorflow.keras.datasets import mnist, fashion_mnist

# Коллбэки для выдачи информации в процессе обучения
from tensorflow.keras.callbacks import LambdaCallback

# Отрисовка изображения
from tensorflow.keras.preprocessing import image

# Разбиение на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split

# Функция среднеквадратической ошибки для расчетов вручную
from sklearn.metrics import mean_squared_error

# Отрисовка графиков
import matplotlib.pyplot as plt

%matplotlib inline
import tensorflow as tf

In [ ]:
# Задание формы изображений с учетом одного канала
shape = 360, 640, 3

# Форма латентного пространства
latent_dim = 8

# ЭНКОДЕР
inputs = Input(shape)
x = Conv2D(8, 3, padding='same', dilation_rate=2, activation='elu')(inputs)
x = BatchNormalization()(x)
x = Conv2D(16, 3, strides=2, padding='same', activation='elu')(x)
x = BatchNormalization()(x)
x = Conv2D(32, 3, strides=2, padding='same', activation='elu')(x)
x = BatchNormalization()(x)
pre_flat_shape = x.shape[1:]
x = Flatten()(x)
flat_shape = x.shape[1]
x = Dense(64, activation='elu')(x)
x = BatchNormalization()(x)
encoder_outputs = Dense(latent_dim)(x)

encoder = Model(inputs, encoder_outputs, name='encoder')

# ДЕКОДЕР
x = encoder_outputs
#x = Concatenate(axis=-1)([x, x ** 2, tf.keras.backend.sqrt(tf.keras.backend.sqrt(x ** 2))])
x = Dense(64, activation='elu')(x)
x = Dense(flat_shape)(x)
x = Reshape(pre_flat_shape)(x)

x = Conv2DTranspose(32, 3, strides=2, padding='same', activation='elu')(x)
x = BatchNormalization()(x)
x = Conv2DTranspose(16, 3, strides=2, padding='same', activation='elu')(x)
x = BatchNormalization()(x)
x = Conv2D(8, 3, padding='same', dilation_rate=2, activation='elu')(x)
x = Conv2D(3, 3, padding='same', activation='sigmoid')(x)
decoder_outputs = x

decoder = Model(inputs, decoder_outputs, name="decoder")

autoencoder = Model(inputs, decoder_outputs, name="autoencoder")

Подгрузим данные для обучения

In [ ]:
import gc
import pickle as pkl
gc.collect()
PATH_VIDEO = 'Internship/Video/orig/'
with open(f'{PATH_VIDEO}1var.pkl', "rb") as f: # Открываем файл для чтения
    train = pkl.load(f)     # Загружаем объект из файла

In [ ]:
train = np.array(train)
train.shape

(1168, 360, 640, 3)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(train[500])

In [ ]:
# Сжатие диапазона [0, 255] значений к диапазону [0, 1]
train = train.astype('float32') / 255.

In [ ]:
from tensorflow.keras.callbacks import Callback, LambdaCallback
import psutil

gc.collect()

def on_epoch_end(self, logs=None):
        memory_usage = round(psutil.Process().memory_info().rss/1024/1024, 2)
        print(f"Объем оперативной памяти перед очиской, занимаемый выполнением кода: {memory_usage} мб")
        gc.collect()
ae_callback = LambdaCallback(on_epoch_end)

In [ ]:
autoencoder.compile(optimizer=Adam(1e-4),
                      loss='mse')

In [ ]:
history = autoencoder.fit(train, train,
                            batch_size=32,
                            epochs=15,
                            validation_split=0.2,
                            verbose=1,
                            callbacks = [ae_callback])


In [ ]:
plt.figure(figsize=(14,7))
plt.plot(history.history['loss'],
        label='Среднеквадратическая ошибка на обучающем наборе')
plt.plot(history.history['val_loss'],
        label='Среднеквадратическая ошибка на проверочном наборе')
plt.ylabel('Средняя ошибка')
plt.legend()
plt.show()

In [ ]:
history = autoencoder.fit(train, train,
                            batch_size=32,
                            epochs=15,
                            validation_split=0.2,
                            verbose=1,
                            callbacks = [ae_callback])


Потенциал обучения еще есть, но на текущие иследования хватит. Сохраним эту модель

In [ ]:
autoencoder.save('Internship/Model/model_var1.h5')
autoencoder.save_weights('Internship/Model/weights_var1.h5')

C:\Users\mrakr\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Модель весит 700 мб. Веса 230 мб. Сохранять в весах имеет смысл.

In [ ]:
train_pred = autoencoder.predict(train[:3])

1/1 [==============================] - 1s 556ms/step


In [ ]:
def show_images(orig, mod, names):
  count = len(orig)
  if (count != len(mod) and count != len(names)):
    print('Массивы должны одинаковое количество элементов')
    return
  figure, axs = plt.subplots(count, 2, figsize=(25, 30))
  for i in range(count):
        axs[i, 0].set_title('оригинальная картинка')
        axs[i, 0].imshow(orig[i])
        axs[i, 0].axis('off')

        axs[i, 1].set_title(names[i])
        axs[i, 1].imshow(mod[i])
        axs[i, 1].axis('off')
  plt.show()

In [ ]:
train_pred[1].shape

(360, 640, 3)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

show_images(train[:3], train_pred[:3], ['предсказание', 'предсказание', 'предсказание', 'предсказание'])

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
# Входной слой
input_img = Input(shape=(shape))

# Энкодер
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Декодер
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# Создание модели автоэнкодера
autoencoder1 = Model(input_img, decoded)